# Salary Prediction — End-to-end Notebook

This notebook performs:
- Data loading & basic cleaning
- Exploratory Data Analysis (EDA)
- Preprocessing pipeline (imputation, encoding, scaling)
- Model training and comparison (Ridge, RandomForest, XGBoost, LightGBM)
- Optional hyperparameter tuning for top candidates
- Residual analysis, feature importance, and SHAP explanations
- Saves the final pipeline to models/best_pipeline.joblib and metadata to models/meta.joblib

Notes:
- This notebook intentionally *excludes* Streamlit web app code. You asked to keep everything in the notebook for easier demonstration.
- If certain packages (xgboost, lightgbm, shap) are missing, either install them in your environment or skip the corresponding cells.

---

## 0. Setup & Imports

In [ ]:
# If you run this in a fresh environment, uncomment and run the next cell to install missing packages:
# !pip install -r requirements.txt

import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint

# sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# optional 3rd party models
try:
    from xgboost import XGBRegressor
except Exception:
    XGBRegressor = None
try:
    from lightgbm import LGBMRegressor
except Exception:
    LGBMRegressor = None

import joblib

plt.style.use("seaborn")
sns.set_context("notebook")

## 1. Load data
Adjust path if your CSV is in another location.

In [ ]:
DATA_PATH = "expected_ctc.csv"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}. Place expected_ctc.csv in the repo root.")

df = pd.read_csv(DATA_PATH, low_memory=False)
print("Initial shape:", df.shape)
df.head(3)

## 2. Target detection & basic cleaning
The dataset sometimes doesn't have a clear target column name. We'll look for common names and otherwise use the last column.

In [ ]:
def detect_target_column(df, candidates=None):
    if candidates is None:
        candidates = ["Expected_CTC", "Expected CTC", "Expected_ctc", "expected_ctc", "CTC", "ctc", "expected", "Expected"]
    for c in candidates:
        if c in df.columns:
            return c
    return df.columns[-1]

target_col = detect_target_column(df)
print("Using target column:", target_col)

# Clean column names
df = df.rename(columns=lambda x: x.strip())

# Replace purely-empty strings with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Make the target numeric and drop rows missing it
df[target_col] = pd.to_numeric(df[target_col], errors="coerce")
n_before = df.shape[0]
df = df.dropna(subset=[target_col]).reset_index(drop=True)
n_after = df.shape[0]
print(f"Dropped {n_before - n_after} rows with missing target")

# Quick target stats
display(df[target_col].describe().astype(int))

# Show skewness (CTC is usually right-skewed)
print("Target skew:", df[target_col].skew())

## 3. Exploratory Data Analysis (suggested / interactive)
Below are useful EDA snippets. Run selectively to inspect relationships.

In [ ]:
# Missing values percent per column (top 40)
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
display(missing_pct.head(40))

In [ ]:
# Visualize target distribution and log-target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
sns.histplot(df[target_col], ax=axes[0], kde=True)
axes[0].set_title("Target distribution")
sns.histplot(np.log1p(df[target_col]), ax=axes[1], kde=True, color="orange")
axes[1].set_title("Log(1+Target) distribution")
plt.show()

In [ ]:
# Check numeric features of interest if they exist (Total_Experience, Total_Experience_in_field_applied)
for col in ["Total_Experience", "Total_Experience_in_field_applied"]:
    if col in df.columns:
        sns.scatterplot(x=col, y=target_col, data=df.sample(2000, random_state=42))
        plt.title(f"{col} vs {target_col}")
        plt.show()

In [ ]:
# Inspect top categorical columns for cardinality and some value counts
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
cat_card = {c: df[c].nunique(dropna=False) for c in cat_cols}
sorted(cat_card.items(), key=lambda x: x[1], reverse=True)[:20]

# Show value counts for a few high-impact columns (if present)
for c in ["Role", "Department", "Industry", "Education", "Designation"]:
    if c in df.columns:
        print("----", c, "----")
        print(df[c].value_counts(dropna=False).head(10))
        print()

Consider log-transforming the target during modeling if error metrics improve (we'll try both approaches later).

## 4. Preprocessing pipeline
We'll build a ColumnTransformer:
- numeric: SimpleImputer(median) + StandardScaler
- categorical: SimpleImputer(constant="Missing") + OneHotEncoder(handle_unknown="ignore")
We'll choose numeric / categorical features heuristically:
  - All numeric dtypes except ID-like columns go to numeric.
  - Object/Category go to categorical.
  - Treat integer columns with small unique values (<=10) as categorical.

In [ ]:
def choose_feature_columns(df, target_col):
    X = df.drop(columns=[target_col])
    numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    cat_from_numeric = [c for c in numeric_cols if X[c].nunique() <= 10]
    numeric_cols = [c for c in numeric_cols if c not in cat_from_numeric]
    categorical_cols = X.select_dtypes(include=["object", "category"]).columns.tolist() + cat_from_numeric
    # remove id-like columns
    id_like = [c for c in X.columns if any(k in c.lower() for k in ["id", "idx", "applicant"]) ]
    categorical_cols = [c for c in categorical_cols if c not in id_like]
    numeric_cols = [c for c in numeric_cols if c not in id_like]
    return numeric_cols, categorical_cols

numeric_cols, categorical_cols = choose_feature_columns(df, target_col)
print("Numeric columns (sample):", numeric_cols[:20])
print("Categorical columns (sample):", categorical_cols[:20])

# Construct preprocessor
numeric_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
categorical_pipeline = Pipeline([("imputer", SimpleImputer(strategy="constant", fill_value="Missing")),
                                 ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False))])

preprocessor = ColumnTransformer([("num", numeric_pipeline, numeric_cols),
                                   ("cat", categorical_pipeline, categorical_cols)],
                                  remainder="drop")

# Quick test: fit_transform small sample to ensure no errors
if len(df) > 0:
    X_sample = df.drop(columns=[target_col]).iloc[:50]
    _ = preprocessor.fit_transform(X_sample)

## 5. Candidate models & cross-validated comparison
We'll train pipelines with:
- Ridge
- RandomForest
- XGBoost (if available)
- LightGBM (if available)

We'll evaluate with 5-fold CV using RMSE, MAE, R2.

In [ ]:
from sklearn.model_selection import KFold

RANDOM_STATE = 42
cv = 5
kf = KFold(n_splits=cv, shuffle=True, random_state=RANDOM_STATE)

candidates = {
    "Ridge": Ridge(random_state=RANDOM_STATE),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1),
}
if XGBRegressor is not None:
    candidates["XGBoost"] = XGBRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1, verbosity=0)
if LGBMRegressor is not None:
    candidates["LightGBM"] = LGBMRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1, verbosity=-1)

results = {}
for name, model in candidates.items():
    print("Evaluating:", name)
    pipe = Pipeline([("preprocessor", preprocessor), ("model", model)])
    scoring = {"rmse": "neg_root_mean_squared_error", "mae": "neg_mean_absolute_error", "r2": "r2"}
    res = cross_validate(pipe, df.drop(columns=[target_col]), df[target_col],
                         scoring=scoring, cv=kf, n_jobs=-1, return_train_score=False)
    rmse = -res["test_rmse"].mean()
    mae = -res["test_mae"].mean()
    r2 = res["test_r2"].mean()
    results[name] = {"rmse": rmse, "mae": mae, "r2": r2}
    print(f"{name} -> RMSE: {rmse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")
    print("-" * 40)

# Summarize results as DataFrame
res_df = pd.DataFrame(results).T.sort_values("rmse")
res_df

If Ridge performs very well compared to tree models, that suggests linear relationships (or good features). If tree models win, we can tune them further.

## 6. (Optional) Try training on log-target and compare
Because target distribution is likely skewed, training on log(1+target) often reduces RMSE. We'll compare best candidate on original target vs log-target.

In [ ]:
from sklearn.base import clone

def cv_score_pipeline(pipeline, X, y, cv):
    scoring = {"rmse": "neg_root_mean_squared_error", "mae": "neg_mean_absolute_error"}
    res = cross_validate(pipeline, X, y, scoring=scoring, cv=cv, n_jobs=-1)
    return {"rmse": -res["test_rmse"].mean(), "mae": -res["test_mae"].mean()}

# choose top candidate from earlier results
best_name = res_df.index[0]
print("Best candidate (original target CV):", best_name)
best_model = candidates[best_name]

# Evaluate best_model with log-target
pipe_orig = Pipeline([("preprocessor", preprocessor), ("model", clone(best_model))])
pipe_log = Pipeline([("preprocessor", preprocessor), ("model", clone(best_model))])

y_log = np.log1p(df[target_col])

print("CV on original target:")
print(cv_score_pipeline(pipe_orig, df.drop(columns=[target_col]), df[target_col], kf))
print("CV on log-target (scores are on log-target):")
print(cv_score_pipeline(pipe_log, df.drop(columns=[target_col]), y_log, kf))

## 7. Fit best pipeline on full data & save artifacts
We'll pick the method we prefer:
- If log-target approach looked better, train on log-target and save an adapter that applies expm1.
For demonstration we'll:
- Use best candidate from earlier (best_name)
- Train on full data (original target)
- Save pipeline to models/best_pipeline.joblib and metadata (residual std, mean) to models/meta.joblib

In [ ]:
os.makedirs("models", exist_ok=True)

final_model = clone(candidates[best_name])
final_pipe = Pipeline([("preprocessor", preprocessor), ("model", final_model)])

print("Fitting final pipeline on all data...")
final_pipe.fit(df.drop(columns=[target_col]), df[target_col])

# Save pipeline
joblib.dump(final_pipe, "models/best_pipeline.joblib")
# Save preprocessor separately too (useful)
joblib.dump(preprocessor, "models/preprocessor.joblib")

# Compute cross-val predictions to estimate residual distribution
print("Computing CV predictions to estimate residual distribution...")
cv_preds = cross_val_predict(final_pipe, df.drop(columns=[target_col]), df[target_col], cv=kf, n_jobs=-1)
residuals = df[target_col] - cv_preds
meta = {"resid_mean": float(residuals.mean()), "resid_std": float(residuals.std()), "cv_rmse": float(mean_squared_error(df[target_col], cv_preds, squared=False))}
joblib.dump(meta, "models/meta.joblib")

print("Saved model pipeline and metadata to models/")

# Quick sanity: predict on first 5 rows
sample_preds = final_pipe.predict(df.drop(columns=[target_col]).iloc[:5])
print("Sample predictions:", sample_preds)

## 8. Residual analysis & simple uncertainty estimate
Use the residual standard deviation to give a simple approximate prediction interval (not a calibrated Bayesian interval).

In [ ]:
preds_all = final_pipe.predict(df.drop(columns=[target_col]))
rmse_full = mean_squared_error(df[target_col], preds_all, squared=False)
mae_full = mean_absolute_error(df[target_col], preds_all)
r2_full = r2_score(df[target_col], preds_all)
print(f"Training-fit metrics -> RMSE: {rmse_full:.0f}, MAE: {mae_full:.0f}, R2: {r2_full:.4f}")
plt.figure(figsize=(8,4))
sns.histplot(df[target_col] - preds_all, bins=60, kde=True)
plt.title("Residual distribution (train-fit)")
plt.show()

# 95% approximate interval on predictions for a candidate:
resid_std = meta["resid_std"]
print("Approx residual std (from CV):", resid_std)
print("Approx 95% PI width ±1.96*std:", 1.96 * resid_std)

## 9. Feature importance & interpretability
- If best model is tree-based, show feature importances.
- If it's linear (Ridge), show coefficients for numeric features and aggregated importance for OHE features.

In [ ]:
def get_feature_names_from_preprocessor(prep, numeric_cols, categorical_cols):
    """Return expanded feature names produced by preprocessor"""
    names = []
    # numeric first
    names.extend(numeric_cols)
    # categorical: get categories_ from onehot encoder
    try:
        # access the onehot encoder inside the column transformer
        for name, transformer, cols in prep.transformers:
            if name == "cat":
                ohe = transformer.named_steps["onehot"]
                cat_cols = list(cols)
                categories = ohe.categories_
                for col, cats in zip(cat_cols, categories):
                    names.extend([f"{col}__{c}" for c in cats])
    except Exception:
        # fallback: use categorical column names only
        names.extend(categorical_cols)
    return names

feature_names = get_feature_names_from_preprocessor(preprocessor, numeric_cols, categorical_cols)
print("Total expanded features (sample):", len(feature_names))
feature_names[:40]

In [ ]:
# Feature importance for tree models
if best_name in ["RandomForest", "XGBoost", "LightGBM"]:
    model = final_pipe.named_steps["model"]
    try:
        importances = model.feature_importances_
        fi = pd.Series(importances, index=feature_names).sort_values(ascending=False).head(40)
        plt.figure(figsize=(8, 10))
        sns.barplot(x=fi.values[:30], y=fi.index[:30])
        plt.title(f"{best_name} feature importances (top 30)")
        plt.show()
    except Exception as e:
        print("Could not compute feature importances:", e)

# Coefficients for Ridge
if best_name == "Ridge":
    coef = final_pipe.named_steps["model"].coef_
    coefs = pd.Series(coef, index=feature_names).sort_values(key=abs, ascending=False).head(40)
    plt.figure(figsize=(8,10))
    sns.barplot(x=coefs.values, y=coefs.index)
    plt.title("Ridge: top coefficient magnitudes")
    plt.show()

## 10. SHAP explanations (optional & recommended)
Compute SHAP summary for a sample if the model supports TreeExplainer (tree models) or KernelExplainer (slower).
Install shap if missing: pip install shap

In [ ]:
try:
    import shap
    shap_available = True
except Exception:
    shap_available = False
    print("shap not installed — skip SHAP cells or install shap to enable them.")

if shap_available:
    # use a sample (to speed up)
    X_full = df.drop(columns=[target_col])
    X_sample = X_full.sample(n=min(500, len(X_full)), random_state=RANDOM_STATE)
    # convert using preprocessor and then use model-specific explainer
    # For convenience, use the pipeline's preprocessor to get numpy array
    X_trans = preprocessor.transform(X_sample)
    model = final_pipe.named_steps["model"]
    explainer = None
    if best_name in ["RandomForest", "XGBoost", "LightGBM"]:
        try:
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_trans)
            # Use feature names already expanded
            shap.summary_plot(shap_values, X_trans, feature_names=feature_names, show=True)
        except Exception as e:
            print("TreeExplainer failed:", e)
            explainer = None
    if explainer is None:
        try:
            # KernelExplainer with a background sample (slow)
            background = X_trans[np.random.choice(X_trans.shape[0], size=min(100, X_trans.shape[0]), replace=False)]
            explainer = shap.KernelExplainer(model.predict, background)
            shap_values = explainer.shap_values(X_trans[:50])
            shap.summary_plot(shap_values, X_trans[:50], feature_names=feature_names)
        except Exception as e:
            print("KernelExplainer failed:", e)


## 11. Save a small evaluation report
Save CV results table and metadata so you can attach in demo.

In [ ]:
report = {"cv_results": results, "cv_summary": res_df.to_dict(), "best_model": best_name, "meta": meta}
joblib.dump(report, "models/report.joblib")
print("Saved models/report.joblib")

# Also save a CSV summary of the comparison
res_df.to_csv("models/model_comparison.csv")
print("Saved models/model_comparison.csv")

## 12. How to use the saved pipeline to predict new candidates
Demonstration code snippet that you can copy into a script or notebook when using saved model.

In [ ]:
# Example:
loaded = joblib.load("models/best_pipeline.joblib")
example_row = df.drop(columns=[target_col]).iloc[0:1]
print("Example input row (first row):")
display(example_row.T.head(30))
pred_example = loaded.predict(example_row)[0]
print("Predicted Expected CTC for example row:", pred_example)

# If you used log-target training, call np.expm1 on the prediction to invert.

## 13. Next steps & tips (for your presentation)
- Inspect `models/model_comparison.csv` and `models/report.joblib` during your demo.
- If you want to show fairness checks:
  - Group by Department/Role/Education and compare median predicted vs actual for similarly-qualified groups.
  - Compute parity metrics (e.g., mean absolute difference between groups).
- For reproducibility on other machines, provide `requirements.txt` and the `models/` artifacts.
- If you'd like I can also:
  - add a cell to generate a small HTML report (pdf) summarizing EDA and evaluation,
  - or produce a condensed Jupyter slideshow (RISE) friendly presentation of the notebook.

This notebook runs the full modeling workflow and stores artifacts in the `models/` directory.